In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
import json
import random
from transformers import AutoTokenizer
from datasets import Dataset

# Load intent data
with open("/content/Intent.json", "r") as file:
    data = json.load(file)

# Prepare conversation pairs
conversations = []
for intent in data["intents"]:
    for text in intent["text"]:
        response = random.choice(intent["responses"])  # Pick a random response
        conversations.append(f"User: {text}\nBot: {response}")

# Load GPT-2 tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Fix: Set padding token explicitly
tokenizer.pad_token = tokenizer.eos_token

# Tokenize conversation data
tokenized_data = tokenizer(conversations, padding=True, truncation=True, return_tensors="pt")

# Convert tokenized data into a Hugging Face Dataset
dataset = Dataset.from_dict({
    "input_ids": tokenized_data["input_ids"],
    "attention_mask": tokenized_data["attention_mask"]
})

# Print a sample to verify everything works
print(dataset[0])

# Now you can train your model with this dataset


{'input_ids': [12982, 25, 23748, 198, 20630, 25, 644, 460, 1312, 466, 329, 345, 30, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 

In [14]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [12]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.0 MB/s eta 0:00:00


In [21]:
import torch
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

model = AutoModelForCausalLM.from_pretrained("gpt2")

tokenized_data = tokenizer(conversations, padding=True, truncation=True, return_tensors="pt")

dataset = Dataset.from_dict({
    "input_ids": tokenized_data["input_ids"],
    "attention_mask": tokenized_data["attention_mask"]
})

training_args = TrainingArguments(
    output_dir="./chatbot_model",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

trainer.train()


Step,Training Loss


TrainOutput(global_step=174, training_loss=2.3790793583310883, metrics={'train_runtime': 1053.3848, 'train_samples_per_second': 0.33, 'train_steps_per_second': 0.165, 'total_flos': 24508376064000.0, 'train_loss': 2.3790793583310883, 'epoch': 1.0})

In [ ]:
def chatbot_response(user_input):
    input_text = f"User: {user_input}\nBot:"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    output = model.generate(input_ids, max_length=50, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

    return response

# Test chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        break
    print("Bot:", chatbot_response(user_input))


You: Asthma symptoms
Bot:  Asthma symptoms include fever, sore throat, and sore throat. Asthma can be managed with regular medical treatments. Asthma can be managed with regular medical check-ups. Asthma may be caused by a
You: what causes malaria
Bot:  it's a common skin condition. It can be managed with proper nutrition, proper nutrition, and regular exercise. It's important to consult a healthcare provider for proper diagnosis and treatment. It's important to consult a
You: what causes dementia
Bot:  you can tell me about it in the comments below.
Bot: I'm sorry for the inconvenience, but I'm not sure how to contact you. I'm sorry to hear that you're suffering from dementia
You: what causes depression
Bot:  depression
Bot: depression is caused by stress, but it can be managed with medication. It may be that you are suffering from a mental illness. It may be that you are suffering from a mental illness.
